# 2. Top N

This notebook shows how to run the Top-N DDA method on a ViMMS dataset

In [1]:
%matplotlib inline

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import sys
sys.path.append('../..')

In [4]:
from pathlib import Path

In [5]:
from vimms.Chemicals import ChemicalCreator
from vimms.MassSpec import IndependentMassSpectrometer
from vimms.Controller import TopNController, PurityController
from vimms.Environment import Environment
from vimms.Common import *

### Load the data

In [6]:
data_dir =  os.path.join(os.path.abspath(os.path.join(os.getcwd(),"..")),'01. Data')
dataset = load_obj(os.path.join(data_dir, 'QCA_22May19_1.p'))
ps = load_obj(os.path.join(data_dir, 'ps.p'))

### Run Top N Controller

In [7]:
rt_range = [(0, 1440)]
min_rt = rt_range[0][0]
max_rt = rt_range[0][1]

In [8]:
isolation_window = 1
N = 3
rt_tol = 15
mz_tol = 10
min_ms1_intensity = 1.75E5

In [9]:
mass_spec = IndependentMassSpectrometer(POSITIVE, dataset, ps)
controller = TopNController(POSITIVE, N, isolation_window, mz_tol, rt_tol, min_ms1_intensity)

In [10]:
# create an environment to run both the mass spec and controller
env = Environment(mass_spec, controller, min_rt, max_rt, progress_bar=True)

# set the log level to WARNING so we don't see too many messages when environment is running
set_log_level_warning()

# run the simulation
env.run()

(1440.053s) ms_level=1 N=3 DEW=15: 100%|████████████████████████████████████████████████████████████████████████████████████████████████▉| 1438.324604670297/1440 [00:10<00:00, 140.10it/s]


Simulated results are saved to the following .mzML file and can be viewed in tools like ToppView or using other mzML file viewers.

In [11]:
set_log_level_debug()
mzml_filename = 'topn_controller.mzML'
env.write_mzML(os.getcwd(), mzml_filename)

2020-07-16 18:46:06.041 | DEBUG    | vimms.Environment:write_mzML:161 - Writing mzML file to C:\Users\Vinny\work\vimms\demo\02. Methods\topn_controller.mzML
2020-07-16 18:46:11.428 | DEBUG    | vimms.Environment:write_mzML:168 - mzML file successfully written!


### Run Top-N Controller with varied N and RT tolerance (DEW)

In [12]:
rt_range = [(0, 1440)]
min_rt = rt_range[0][0]
max_rt = rt_range[0][1]

In [13]:
ionisation_mode = POSITIVE
isolation_window = [1, 1.1, 1.2, 1.3]
N = [5, 10, 15, 20]
rt_tol = [15, 30, 60, 120]
mz_tol = [10, 5, 15, 20]
scan_param_changepoints = [300, 600, 900]

In [14]:
mass_spec = IndependentMassSpectrometer(POSITIVE, dataset, ps)
controller = PurityController(POSITIVE, N, scan_param_changepoints, isolation_window, mz_tol, rt_tol, min_ms1_intensity)

In [15]:
# create an environment to run both the mass spec and controller
env = Environment(mass_spec, controller, min_rt, max_rt, progress_bar=True)

# set the log level to WARNING so we don't see too many messages when environment is running
set_log_level_warning()

# run the simulation
env.run()

(1440.162s) ms_level=1 N=20 DEW=120: 100%|█████████████████████████████████████████████████████████████████████████████████████████████▉| 1438.4216635970445/1440 [00:10<00:00, 133.85it/s]


Simulated results are saved to the following .mzML file and can be viewed in tools like ToppView or using other mzML file viewers.

In [16]:
set_log_level_debug()
mzml_filename = 'topnvaried_controller.mzML'
env.write_mzML(os.getcwd(), mzml_filename)

2020-07-16 18:46:22.743 | DEBUG    | vimms.Environment:write_mzML:161 - Writing mzML file to C:\Users\Vinny\work\vimms\demo\02. Methods\topnvaried_controller.mzML
2020-07-16 18:46:26.360 | DEBUG    | vimms.Environment:write_mzML:168 - mzML file successfully written!
